## Pretrained model
- tensorflow hub: https://www.tensorflow.org/hub
- pytorch hub: https://pytorch.org/hub/
- 노트북에서 실행시간 매우 길어 실습 어려움

In [1]:
# 필요한 라이브러리 임포트
import torch  # PyTorch 텐서 및 연산
import torch.nn as nn  # 신경망 레이어/모듈 (Linear, Conv 등) 및 손실 함수
import torch.optim as optim  # 최적화 알고리즘 (SGD, Adam 등)
from torch.utils.data import DataLoader, TensorDataset  # 데이터 로딩 유틸 및 텐서 기반 데이터셋
import numpy as np  # 수치 연산, 난수 생성 등에 사용
from torchvision import models, transforms  # 사전학습(Pretrained) 모델과 이미지 전처리 도구

In [2]:
# 데이터 설정 및 더미 데이터 생성
num_classes = 10  # 분류할 클래스 개수
input_shape = (3, 224, 224)  # EfficientNetB0의 기본 입력 크기 (채널=3, 224x224)

# 데모를 위한 난수 입력/레이블 생성 (실전에서는 실제 이미지와 라벨을 사용)
x_train = np.random.random((1000, 3, 224, 224)).astype(np.float32)  # 1,000개 학습용 이미지
y_train = np.random.randint(num_classes, size=(1000,))  # 0~9 사이의 정수 라벨
x_test = np.random.random((200, 3, 224, 224)).astype(np.float32)   # 200개 테스트용 이미지
y_test = np.random.randint(num_classes, size=(200,))  # 테스트 라벨

# 텐서로 변환하여 TensorDataset 구성 (이미지가 이미 텐서 형태이므로 추가 변환 불필요)
train_dataset = TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
test_dataset = TensorDataset(torch.tensor(x_test), torch.tensor(y_test))

# DataLoader로 배치 단위 로딩 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)   # 학습 데이터는 섞음
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)    # 테스트 데이터는 순서 유지

# 참고: 실제 이미지 데이터를 사용할 때는 transforms.Resize(224), transforms.Normalize 등 전처리 필요

In [3]:
# 사전훈련 모델 불러오기 및 출력층 수정
# EfficientNetB0 (ImageNet으로 사전훈련된 가중치 사용)
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# 분류기(마지막 Linear 레이어)를 현재 데이터셋의 클래스 수에 맞게 교체 (1000 → num_classes)
# EfficientNetB0의 classifier는 [Dropout, Linear] 구조이므로 index 1의 Linear를 교체
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# 참고:
# - 전이학습에서 feature extractor를 고정하고 싶다면, 다음처럼 requires_grad를 끌 수 있음
#   for param in model.features.parameters():
#       param.requires_grad = False
# - 입력 텐서는 (N, 3, 224, 224) 형식이어야 하며, ImageNet 정규화로 학습된 모델이므로
#   실제 이미지에서는 transforms.Normalize(mean, std)를 적용해야함

In [4]:
# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류에 일반적으로 사용하는 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저 (작은 학습률 권장)

In [ ]:
# 모델 학습 루프 (간단 버전) 
# 노트북에서 실행시간 매우 길어 실습 어려움

num_epochs = 3  # 사전훈련 모델은 빠르게 수렴하는 경향이 있어 데모에서는 짧게 설정

for epoch in range(num_epochs):
    model.train()  # 학습 모드로 전환 (Dropout/BatchNorm 활성화 동작)
    running_loss = 0.0

    for inputs, labels in train_loader:
        # (옵션) GPU 사용 시: inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()      # 이전 배치의 기울기 초기화
        outputs = model(inputs)    # 순전파(Forward): 예측값 계산
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()            # 역전파(Backward): 기울기 계산
        optimizer.step()           # 가중치 업데이트

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}')

In [ ]:
# 모델 평가 (정확도 측정)
model.eval()  # 평가 모드로 전환 (Dropout/BatchNorm 비활성화 동작)
correct = 0
total = 0

with torch.no_grad():  # 평가 시 기울기 계산 비활성화 (메모리/속도 이점)
    for inputs, labels in test_loader:
        # (옵션) GPU 사용 시: inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)             # 예측값 계산
        _, predicted = torch.max(outputs, 1)  # 각 샘플의 최대 점수 클래스 선택
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')